## AI Agent with Pydantic to get the Approximate price of the Product in $$

- Create a simple assistant that uses any LLM and it should use pydantic
- When we ask about any product it should give you two information of product Name, product details tentative price in USD (integer). 
- Use chat Prompt Template.

In [61]:
# Import libraries
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from dataclasses import dataclass

# GROQ Chat LLM
from langchain_groq import ChatGroq

# Chat Prompt Template
from langchain_core.prompts import ChatPromptTemplate

from langchain.output_parsers import PydanticOutputParser


In [69]:
# Create Pydantic Product class
@dataclass
class Product(BaseModel):
    product_name:str=Field(description="Name of the Product")
    description:str=Field(description="Product Description")
    price:float=Field(description="Price of the Product")
    website:str=Field(description="Website URL")

In [ ]:
# Load local environment
load_dotenv()

# Set environment variables
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

## LangSmith Tracking & Tracing
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

#### Determine the model to use

In [65]:
# LLM Model to use
llm_groq_gemma = ChatGroq(model="llama-3.1-8b-instant")
print(llm_groq_gemma)

client=<groq.resources.chat.completions.Completions object at 0x316464800> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x3169a0d40> model_name='llama-3.1-8b-instant' model_kwargs={} groq_api_key=SecretStr('**********')


In [70]:
# Create the parser
parser = PydanticOutputParser(pydantic_object=Product)

# Create prompt template with format instructions
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a product information specialist. Extract or generate product details in a structured format."),
    ("human", """
    Please provide product information for: {query}
    
    {format_instructions}
    """)
])

# Create the chain
chain = prompt | llm_groq_gemma | parser

# Test it
response = chain.invoke({
    "query": "iPhone 15 Pro Max",
    "format_instructions": parser.get_format_instructions()
})

In [71]:
result = chain.invoke({
    "query": "Tell me about the MacBook Pro 16-inch",
    "format_instructions": parser.get_format_instructions()
})

print(f"Product Name: {result.product_name}")
print(f"Description: {result.description}")
print(f"Price: ${result.price}")
print(f"Website: ${result.website}")

Product Name: MacBook Pro 16-inch
Description: The MacBook Pro 16-inch is a high-performance laptop featuring a 16-inch Retina display, up to 64GB of RAM, and up to 8TB of storage. It's powered by Apple's M1 or M2 chip and features a range of ports including Thunderbolt 3, HDMI, and an SD card slot.
Price: $2399.99
Website: $https://www.apple.com/macbook-pro/


In [72]:
result = chain.invoke({
    "query": "Tell me about the Samsung tv 65 inch 4K",
    "format_instructions": parser.get_format_instructions()
})

print(f"Product Name: {result.product_name}")
print(f"Description: {result.description}")
print(f"Price: ${result.price}")
print(f"Website: ${result.website}")

Product Name: Samsung 65 inch 4K TV
Description: Experience stunning 4K resolution and vibrant colors with this 65 inch Samsung TV. Enjoy a wide range of entertainment options, including streaming services and built-in apps.
Price: $999.99
Website: $https://www.samsung.com/us/
}
```

**Product Details:**

* **Product Name**: Samsung 65 inch 4K TV
* **Description**: Experience stunning 4K resolution and vibrant colors with this 65 inch Samsung TV. Enjoy a wide range of entertainment options, including streaming services and built-in apps.
* **Price**: $999.99
* **Website**: https://www.samsung.com/us/ (Note: This website URL may be subject to change)

**Technical Specifications:**

* Display Size: 65 inches
* Resolution: 3840 x 2160 (4K)
* Panel Type: LED
* Refresh Rate: 120Hz
* HDR Support: Yes (HDR10+)
* Smart TV Platform: Tizen
* Connectivity: 4 HDMI ports, 2 USB ports, Wi-Fi, Ethernet

**Additional Features:**

* Ambient Mode: Yes
* Voice Control: Yes (via Samsung Bixby)
* Multi-Dev